# Where to take driving exam? Candidate number: 50691
<hr>

## Objectives:
- Expected passing for XYZ in both locations.
- Where should XYZ take the test? Statistical evidence in favour of the decision?
- Justify the methodology used, explain assumptions.
- Final report needs to be understood by non-expert in statistics.
- Commented code, so that it can be understood.
- Strenghts and weaknesses, relate to assumptions I make. Also maybe mention missing data, XYZ thinks his/her driving skill is about average.
- <b>8 pages max, can be shorter</b>


<hr> 

In [39]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [7]:
ID <- 202215485
source("XYZprofile.r")
XYZprofile(ID)

The profile of XYZ:
- Age:  21
- Gender:  Female
- Home address:  Reading


## Exploratory data analysis:
    This should follow a purpose, which is mostly sample selection:
- Passing rate is higher in Reading
- Similar sample size in both locations.
- Large differences between males and females, they do not come from same distribution. -> Important for tests I guess.

<b>Can we bunch together different ages, gender and years<b>

Gender is immediately discarded as there are large differences. 

<b>To do:<b>

- Check differences in passing rates for women of a different year, for all years.
- If they are somewhat constant for a decent enough sample size. -> We can bunch women together from 20-23, see the slope.
- If we can bunch ages together maybe time series plot becomes a bit smoother and we can bunch different years together.

<i>On the other hand, if even when bunching time series plot is not smooth we can't use that, so start with time series bunched. If there is a lot of variability we just take latest year, and we just we can just check that year out.<i>

<b>Confirmed time series is not too smooth, for now cut sample to most recent year. AND, within that year it is not too smooth so for test restrict the sample. To just women 21</b>


This reasoning should be a bit better.

In [66]:
passing_rates_reading <- read.csv("passing_rates_reading.csv")

In [68]:
head(passing_rates_reading,5)

,Location,Age,Conducted,Passes,Pass.rate....,gender,year
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<chr>
1,Reading,17,307,193,62.86645,0,2023-24
2,Reading,18,366,181,49.45355,0,2023-24
3,Reading,19,216,116,53.70370,0,2023-24
4,Reading,20,147,81,55.10204,0,2023-24
5,Reading,21,125,68,54.40000,0,2023-24


## Logit can be a great option
<b>Especially for conditional probability estimation<b>

Remember for confidence interval quantify the epsilon uncertainty, can be an interesting fact. Is this valid with logit or not? That tutorial from ML class.

## Tests, check out all of statistics chapter.

We can start from most restrictive sample scenario, compare only women aged 21 in last year, if we discard with smallest sample we will discard with bigger sample.

To do list:

- Derivation of the fact that difference of two sample means follow an asymptotically normal distribution. MLE justification.

- Talk about the power of the Wald test. If we increase the population size the test size would increase. 

<hr>

### Derivation of Wald test.
<b>Assume</b> that all 21 year old women have the same preparation and driving skill.

Then we can say going into exam $R \sim Binomial(N_r,p_r)$ and $Y \sim Binomial(N_l,p_l)$

That is every the probability that someone passes the exam is independent with the probability someone else passes it. 

<b>Assume</b> that the two binomials are independent, seems straightforward but there can be some selection issues and it is tough to formalize.

Then we can set up an MLE estimator which will be smooth over parameter space so it will be asymptotically normal: $f(x,y) =f(x)f(y)$ taking logs means that p1 and p2 will be estimated independently. To get to $f(x)f(y)$ we have used the independence.

Then we have a very nice theorem: 9.14 Theorem. Let $\tau = g(\theta)$ be a function of $\theta$. Let $\hat{\theta}$ be the MLE of $\theta$. Then $\hat{\tau} = g(\hat{\theta})$ is the MLE of $\tau$.

In our case the function is $g(p_r,p_l) = p_r - p_l$ so our MLE will just be the plug in. 
For the variance we can derive it easily through independence $\textbf{V}(\hat{p_r}) + \textbf{V}(\hat{p_l})$

In [86]:
size_r <- 98
prob_r <- 49/size_r

size_l <- 310
prob_l <- 135/size_l

In [87]:
prob_r - prob_l

[1] 0.06451613

In [88]:
se_mle <- sqrt( (prob_r)*(1-prob_r)/size_r + (prob_l)*(1-prob_l)/size_l )

In [89]:
W <- (prob_r-prob_l)/se_mle
W

[1] 1.115661

In [90]:
abs(W) > 1.645

[1] FALSE

Under the most strict assumptions we can't discard the null, we either need additional data or a more powerful test.
Add power add sample! -> 21-23 a bit arbitrary on who we include but it seems similar enough! 

In [55]:
p_rate_age_subset <- subset(passing_rates_reading, Age %in% c(21,22,23))



In [56]:
p_rate_age_subset <- p_rate_age_subset %>% group_by(gender, year, Location) %>% 
  summarise(conducted=sum(
      Conducted), passes = sum(Passes)
           )


`summarise()` has grouped output by 'gender', 'year'. You can override using
the `.groups` argument.


In [74]:
p_rate_age_subset <- data.frame(subset(p_rate_age_subset, year=="2023-24"))

In [79]:
p_rate_age_subset$rate <- p_rate_age_subset$passes / p_rate_age_subset$conducted

In [80]:
p_rate_age_subset

gender,year,Location,conducted,passes,rate
<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
0,2023-24,Reading,353,184,0.5212465
0,2023-24,Wood Green (London),1005,473,0.4706468
1,2023-24,Reading,287,133,0.4634146
1,2023-24,Wood Green (London),855,359,0.4198830


Figure out how to get the numbers out of the dataframe!

In [91]:
size_r <-287
prob_r <- 133/size_r

size_l <- 855
prob_l <- 359/size_l

In [92]:
se_mle <- sqrt( (prob_r)*(1-prob_r)/size_r + (prob_l)*(1-prob_l)/size_l )

In [93]:
W <- (prob_r-prob_l)/se_mle

In [94]:
W

[1] 1.282948

Still not enough power!!!

### Permutation test:
Can be used to compare two means even though I don't fully know why!
We have an initial test statistic, we call it t observed.

Make up a random vector even though we do not observe it; (r_1: 1, r_2: 0, r_3: 1, l_1:1 , ... l_n=1) such that the averages sum up to observed passing rates.
Then use random to shuffle this list: argue that same permutation will rarely happen. 
Shuffle and get 1:n_r as the reading obs rest are london.
Compute the statistic and see if it is larger to observed. 1 if so.
Then p value is just sum of ones divided by iterations we take.



In [59]:
permutation_test <- function(total_r, pass_r, total_l, pass_l, iter){
    
    sample_r <- c(numeric(pass_r)+1, numeric(total_r-pass_r))
    sample_l <- c(numeric(pass_l)+1, numeric(total_l-pass_l))

    full_sample <- c(sample_r, sample_l)
    
    original_statistic <- mean(sample_r) - mean(sample_l)

    larger_than_og <-c()
    
    for (i in 1:iter){
        permutation <- sample(full_sample) # Sampled without replacement.
        # May have to fix it so no vectors are duplicated, I feel the probability is super low anyways.
        
        perm_r <- permutation[1:total_r]
        perm_l <- permutation[total_r=1:length(permutation)]

        perm_statistic <- mean(perm_r) - mean(perm_l)

        larger_than_og <- c(larger_than_og, perm_statistic > original_statistic)
        
    } 
    
    return(mean(larger_than_og))
}    

In [64]:
# Even with most restricted sample we have this with very good power.

permutation_test(98,49,310,135,10000)

[1] 0.0693

In [60]:
# Permutation test destroying the Wald.

permutation_test(287,133,855,359,10000)

[1] 0.0409

### Parametric bootstrap can be used to test as well no?

Initially thought of it for the plots but can be implemented here as well.

We have derived our MLE estimator so parametric bootstrap is here just to draw from Binomial with the plug in estimator which is the sample average.

Talk about relationship between confidence interval and testing.

In [35]:
boot_s <- 100000

In [3]:
#  women 21 for latest year.
#310.0	135.0	
# 98.0	49.0


In [15]:
set.seed(100)
boot_prob_r <- rbinom(boot_s, size_r, prob_r)/size_r
boot_prob_l <- rbinom(boot_s, size_l, prob_l)/size_l


In [21]:
#Smallish sample.

quantile(boot_prob_l,0.025)
quantile(boot_prob_l,0.975)


quantile(boot_prob_r,0.025)
quantile(boot_prob_r,0.975)

2.5% 
0.3806452

97.5% 
0.4903226

2.5% 
0.3979592

97.5% 
0.6020408